###### In this assignment students need to predict whether a person makes over 50K per year or not from classic adult dataset using XGBoost.
###### trainig DataSet- http://archive.ics.uci.edu/ml/machine-learningdatabases/adult/adult.data
###### test DataSet- http://archive.ics.uci.edu/ml/machine-learningdatabases/adult/adult.test

In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb

In [11]:
train_set = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header = None)
test_set = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test',skiprows = 1, header = None)


In [12]:
print("Train_shape %:",(train_set.shape[0]/(train_set.shape[0]+test_set.shape[0]))*100)
print("Test_shape % :",(test_set.shape[0]/(train_set.shape[0]+test_set.shape[0]))*100)

Train_shape %: 66.66598419393145
Test_shape % : 33.33401580606854


In [14]:
combined=pd.concat([train_set,test_set],axis=0)

In [15]:
print("Train_shape :",combined.shape)

Train_shape : (48842, 15)


In [16]:
combined.rename(columns={0:'age',1:'workclass',2:'fnlwgt',3:'education',4:'education_num',5:'marital_status',6:'occupation',7:'relationship',8:'race',9:'sex',10:'capital_gain',11:'capital_loss',12:'hours_per_week',13:'native_country',14:'wage_class'},inplace=True)
combined.head(2)


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K


In [66]:
set(combined['wage_class'])

{' <=50K', ' <=50K.', ' >50K', ' >50K.'}

In [67]:
combined['wage_class'] = combined['wage_class'].map({' <=50K':'<=50K',' <=50K.':'<=50K' ,' >50K': '>50K',' >50K.': '>50K'})
#combined['wage_class'] = combined['wage_class'].map({' <=50K.':'<=50K', ' >50K.': '>50K'})

In [68]:
set(combined['wage_class'])

{'<=50K', '>50K'}

In [69]:
combined.replace(' ?', np.nan).dropna().shape

(45222, 15)

In [70]:
combined.workclass.replace(' ?',None,inplace=True)
combined.occupation.replace(' ?',None,inplace=True)
combined.native_country.replace(' ?',None,inplace=True)

In [72]:
#work class train data into categorical clolumn
wrkcls_mask=pd.get_dummies(combined['workclass'],prefix='wrk').iloc[:,1:]
combined= pd.concat([combined,wrkcls_mask],axis=1)
del combined['workclass']

In [73]:
#Educaion class train data into categorical clolumn
edu_mask=pd.get_dummies(combined['education'],prefix='edu').iloc[:,1:]
combined=pd.concat([combined,edu_mask],axis=1)
del combined['education']

In [74]:
#marital status class train data into categorical clolumn
ms_mask=pd.get_dummies(combined['marital_status'],prefix='mar_stat').iloc[:,1:]
combined=pd.concat([combined,ms_mask],axis=1)
del combined['marital_status']

In [75]:
#Occupation class train data into categorical clolumn
ocu_mask=pd.get_dummies(combined['occupation'],prefix='occu').iloc[:,1:]
combined=pd.concat([combined,ocu_mask],axis=1)
del combined['occupation']

In [76]:
#Relationship class train data into categorical clolumn
rel_mask=pd.get_dummies(combined['relationship'],prefix='reln').iloc[:,1:]
combined=pd.concat([combined,rel_mask],axis=1)
del combined['relationship']

In [77]:
#Race class train data into categorical clolumn
race_mask=pd.get_dummies(combined['race'],prefix='race').iloc[:,1:]
combined=pd.concat([combined,race_mask],axis=1)
del combined['race']

In [78]:
#Sex class train data into categorical clolumn
sex_mask=pd.get_dummies(combined['sex'],prefix='sex').iloc[:,1:]
combined=pd.concat([combined,sex_mask],axis=1)
del combined['sex']

In [79]:
#Native country class test data into categorical clolumn
nc_mask=pd.get_dummies(combined['native_country'],prefix='country').iloc[:,1:]
combined=pd.concat([combined,nc_mask],axis=1)
del combined['native_country']

In [97]:
combined.shape

(48842, 98)

In [107]:
train=combined[:31747]
test=combined[31747:]

48842

In [108]:
y_train= train.pop('wage_class')
x_train= train
y_test= test.pop('wage_class')
x_test= test

In [109]:
model1=xgb.XGBClassifier()

In [111]:
model1.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [114]:
model1.score(x_train,y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8670740542413456

In [115]:
model1.score(x_test,y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8680315881836794